In [ ]:
%pip install geopandas

In [13]:
import pandas as pd 
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html



In [14]:
df= pd.read_csv('C:/Users/Admin/Downloads\DIRISA TB/NewSt/data/number-of-deaths-from-tuberculosis-ghe.csv')

# Preprocess Data


In [15]:
# Rename columns for simplicity
df.rename(columns={
    'Entity': 'Country',
    'Code': 'Country_Code',
    'Year': 'Year',
    'Total deaths from tuberculosis among both sexes': 'TB_Deaths'
}, inplace=True)

# Ensure Year is an integer and TB_Deaths is numeric
df['Year'] = pd.to_datetime(df['Year'], format='%Y').dt.year
df['TB_Deaths'] = pd.to_numeric(df['TB_Deaths'], errors='coerce')

# Preview the data
print(df.head())


       Country Country_Code  Year  TB_Deaths
0  Afghanistan          AFG  2000   13343.42
1  Afghanistan          AFG  2001   12493.19
2  Afghanistan          AFG  2002   12069.03
3  Afghanistan          AFG  2003   13053.80
4  Afghanistan          AFG  2004   12078.29


# Visualize on Dashboard

In [19]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Initialize Dash app
app = Dash(__name__)

# Define filter ranges
filter_ranges = [
    'No Data', '0-300', '300-1,000', '1,000-3,000',
    '3,000-10,000', '10,000-30,000', '30,000-100,000', '100,000+'
]

# Layout of the dashboard
app.layout = html.Div([
    html.H1(
        'Global Tuberculosis Dashboard', 
        style={'textAlign': 'center', 'color': '#4CAF50', 'font-family': 'Arial, sans-serif'}
    ),
    
    html.Div([
        html.Label("Select Countries:", style={'font-size': '16px'}),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in df['Country'].unique()],
            multi=True,
            placeholder='Select countries...',
            style={'margin-bottom': '15px'}
        ),

        html.Label("Filter by TB Deaths:", style={'font-size': '16px'}),
        dcc.Dropdown(
            id='range-dropdown',
            options=[{'label': r, 'value': r} for r in filter_ranges],
            placeholder='Filter by TB Deaths range...',
            style={'margin-bottom': '15px'}
        )
    ], style={'width': '50%', 'margin': '0 auto'}),
    
    dcc.Graph(id='time-lapse-map', style={'margin-top': '20px'}),
    dcc.Graph(id='line-graph', style={'margin-top': '20px'}),
])

# Callback for interactive updates
@app.callback(
    [Output('time-lapse-map', 'figure'),
     Output('line-graph', 'figure')],
    [Input('country-dropdown', 'value'),
     Input('range-dropdown', 'value')]
)
def update_graphs(selected_countries, selected_range):
    # Filter dataset
    filtered_df = df.copy()
    if selected_countries:
        filtered_df = filtered_df[filtered_df['Country'].isin(selected_countries)]

    if selected_range:
        if selected_range == 'No Data':
            filtered_df = filtered_df[filtered_df['TB_Deaths'].isnull()]
        else:
            bounds = selected_range.replace('+', '').split('-')
            lower = int(bounds[0])
            upper = float('inf') if len(bounds) == 1 else int(bounds[1])
            filtered_df = filtered_df[
                (filtered_df['TB_Deaths'] >= lower) & 
                (filtered_df['TB_Deaths'] <= upper)
            ]

    # Choropleth map
    map_fig = px.choropleth(
        filtered_df,
        locations='Country_Code',
        locationmode='ISO-3',
        color='TB_Deaths',
        hover_name='Country',
        animation_frame='Year',
        title='Time-Lapse of Tuberculosis Deaths Worldwide',
        color_continuous_scale=[
            "#f7fbff", "#deebf7", "#c6dbef", "#9ecae1",
            "#6baed6", "#4292c6", "#2171b5", "#084594"
        ],
        range_color=[0, df['TB_Deaths'].max()]
    )
    map_fig.update_layout(
        geo=dict(showframe=False, showcoastlines=False, projection_type='natural earth'),
        title_font=dict(size=24, family='Arial', color='#4CAF50'),
    )

    # Line graph with vertical markers
    line_fig = go.Figure()
    for country in filtered_df['Country'].unique():
        country_data = filtered_df[filtered_df['Country'] == country]
        line_fig.add_trace(go.Scatter(
            x=country_data['Year'],
            y=country_data['TB_Deaths'],
            mode='lines+markers',
            name=country,
            line=dict(width=2),
        ))

    # Add vertical line markers for significant years (optional example: 2000, 2010, 2020)
    for year in [2000, 2010, 2020]:
        line_fig.add_vline(
            x=year, 
            line=dict(color='gray', dash='dash', width=1), 
            annotation_text=f'Year {year}', 
            annotation_position='top'
        )

    line_fig.update_layout(
        title='Trends in Tuberculosis Deaths Over Time',
        title_font=dict(size=22, family='Arial', color='#4CAF50'),
        xaxis_title='Year',
        yaxis_title='Total TB Deaths',
        template='plotly_white',
        legend_title='Country',
        font=dict(family='Arial', size=14),
    )

    return map_fig, line_fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


In [18]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Initialize Dash app
app = Dash(__name__)

# Define filter ranges
filter_ranges = [
    'No Data', '0-300', '300-1,000', '1,000-3,000',
    '3,000-10,000', '10,000-30,000', '30,000-100,000', '100,000+'
]

# Layout of the dashboard
app.layout = html.Div([
    html.H1(
        'Global Tuberculosis Dashboard', 
        style={'textAlign': 'center', 'color': '#4CAF50', 'font-family': 'Arial, sans-serif'}
    ),
    
    html.Div([
        html.Label("Select Countries:", style={'font-size': '16px'}),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in df['Country'].unique()],
            multi=True,
            placeholder='Select countries...',
            style={'margin-bottom': '15px'}
        ),

        html.Label("Filter by TB Deaths:", style={'font-size': '16px'}),
        dcc.Dropdown(
            id='range-dropdown',
            options=[{'label': r, 'value': r} for r in filter_ranges],
            placeholder='Filter by TB Deaths range...',
            style={'margin-bottom': '15px'}
        )
    ], style={'width': '50%', 'margin': '0 auto'}),
    
    dcc.Graph(id='time-lapse-map', style={'margin-top': '20px'}),
    dcc.Graph(id='line-graph', style={'margin-top': '20px'}),
])

# Callback for interactive updates
@app.callback(
    [Output('time-lapse-map', 'figure'),
     Output('line-graph', 'figure')],
    [Input('country-dropdown', 'value'),
     Input('range-dropdown', 'value')]
)
def update_graphs(selected_countries, selected_range):
    # Filter dataset
    filtered_df = df.copy()
    if selected_countries:
        filtered_df = filtered_df[filtered_df['Country'].isin(selected_countries)]

    if selected_range:
        if selected_range == 'No Data':
            filtered_df = filtered_df[filtered_df['TB_Deaths'].isnull()]
        else:
            bounds = selected_range.replace('+', '').split('-')
            lower = int(bounds[0])
            upper = float('inf') if len(bounds) == 1 else int(bounds[1])
            filtered_df = filtered_df[
                (filtered_df['TB_Deaths'] >= lower) & 
                (filtered_df['TB_Deaths'] <= upper)
            ]

    # Choropleth map
    map_fig = px.choropleth(
        filtered_df,
        locations='Country_Code',
        locationmode='ISO-3',
        color='TB_Deaths',
        hover_name='Country',
        animation_frame='Year',
        title='Time-Lapse of Tuberculosis Deaths Worldwide',
        color_continuous_scale=[
            "#f7fbff", "#deebf7", "#c6dbef", "#9ecae1",
            "#6baed6", "#4292c6", "#2171b5", "#084594"
        ],
        range_color=[0, df['TB_Deaths'].max()]
    )
    map_fig.update_layout(
        geo=dict(showframe=False, showcoastlines=False, projection_type='natural earth'),
        title_font=dict(size=24, family='Arial', color='#4CAF50'),
    )

    # Line graph with vertical markers
    line_fig = go.Figure()
    for country in filtered_df['Country'].unique():
        country_data = filtered_df[filtered_df['Country'] == country]
        line_fig.add_trace(go.Scatter(
            x=country_data['Year'],
            y=country_data['TB_Deaths'],
            mode='lines+markers',
            name=country,
            line=dict(width=2),
        ))

    # Add vertical line markers for significant years (optional example: 2000, 2010, 2020)
    for year in [2000, 2010, 2020]:
        line_fig.add_vline(
            x=year, 
            line=dict(color='gray', dash='dash', width=1), 
            annotation_text=f'Year {year}', 
            annotation_position='top'
        )

    line_fig.update_layout(
        title='Trends in Tuberculosis Deaths Over Time',
        title_font=dict(size=22, family='Arial', color='#4CAF50'),
        xaxis_title='Year',
        yaxis_title='Total TB Deaths',
        template='plotly_white',
        legend_title='Country',
        font=dict(family='Arial', size=14),
    )

    return map_fig, line_fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


In [20]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go




# Define filter ranges
filter_ranges = [
    'No Data', '0-300', '300-1,000', '1,000-3,000',
    '3,000-10,000', '10,000-30,000', '30,000-100,000', '100,000+'
]

# Initialize Dash app
app = Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1(
        'Global Tuberculosis Dashboard',
        style={'textAlign': 'center', 'color': '#4CAF50', 'font-family': 'Arial, sans-serif'}
    ),
    html.Div([
        html.Label("Select Countries:", style={'font-size': '16px'}),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in df['Country'].unique()],
            multi=True,
            placeholder='Select countries...',
            style={'margin-bottom': '15px'}
        ),
        html.Label("Filter by TB Deaths:", style={'font-size': '16px'}),
        dcc.Dropdown(
            id='range-dropdown',
            options=[{'label': r, 'value': r} for r in filter_ranges],
            placeholder='Filter by TB Deaths range...',
            style={'margin-bottom': '15px'}
        )
    ], style={'width': '50%', 'margin': '0 auto'}),
    dcc.Graph(id='time-lapse-map', style={'margin-top': '20px'}),
    dcc.Graph(id='line-graph', style={'margin-top': '20px'}),
])

# Callback for interactive updates
@app.callback(
    [Output('time-lapse-map', 'figure'),
     Output('line-graph', 'figure')],
    [Input('country-dropdown', 'value'),
     Input('range-dropdown', 'value')]
)
def update_graphs(selected_countries, selected_range):
    # Filter dataset
    filtered_df = df.copy()
    if selected_countries:
        filtered_df = filtered_df[filtered_df['Country'].isin(selected_countries)]

    if selected_range:
        if selected_range == 'No Data':
            filtered_df = filtered_df[filtered_df['TB_Deaths'].isnull()]
        else:
            bounds = selected_range.replace('+', '').split('-')
            lower = int(bounds[0])
            upper = float('inf') if len(bounds) == 1 else int(bounds[1])
            filtered_df = filtered_df[
                (filtered_df['TB_Deaths'] >= lower) & 
                (filtered_df['TB_Deaths'] <= upper)
            ]

    # Define colors for each TB death range
    color_mapping = {
        'No Data': '#d3d3d3',  # Grey
        '0-300': '#fee5d9',    # Light Red
        '300-1,000': '#fcae91', # Pink
        '1,000-3,000': '#fb6a4a', # Red
        '3,000-10,000': '#de2d26', # Dark Red
        '10,000-30,000': '#a50f15', # Burgundy
        '30,000-100,000': '#67000d', # Maroon
        '100,000+': '#49006a'  # Purple
    }

    # Add a categorical column for TB death ranges
    def get_range(value):
        if pd.isna(value):
            return 'No Data'
        elif value < 300:
            return '0-300'
        elif value < 1000:
            return '300-1,000'
        elif value < 3000:
            return '1,000-3,000'
        elif value < 10000:
            return '3,000-10,000'
        elif value < 30000:
            return '10,000-30,000'
        elif value < 100000:
            return '30,000-100,000'
        else:
            return '100,000+'

    filtered_df['TB_Death_Range'] = filtered_df['TB_Deaths'].apply(get_range)

    # Create the choropleth map with discrete colors
    map_fig = px.choropleth(
        filtered_df,
        locations='Country_Code',
        locationmode='ISO-3',
        color='TB_Death_Range',
        hover_name='Country',
        animation_frame='Year',
        title='Time-Lapse of Tuberculosis Deaths Worldwide',
        category_orders={"TB_Death_Range": list(color_mapping.keys())},
        color_discrete_map=color_mapping
    )
    map_fig.update_layout(
        geo=dict(showframe=False, showcoastlines=False, projection_type='natural earth'),
        title_font=dict(size=24, family='Arial', color='#4CAF50'),
    )

    # Line graph with vertical markers (unchanged)
    line_fig = go.Figure()
    for country in filtered_df['Country'].unique():
        country_data = filtered_df[filtered_df['Country'] == country]
        line_fig.add_trace(go.Scatter(
            x=country_data['Year'],
            y=country_data['TB_Deaths'],
            mode='lines+markers',
            name=country,
            line=dict(width=2),
        ))

    line_fig.update_layout(
        title='Trends in Tuberculosis Deaths Over Time',
        title_font=dict(size=22, family='Arial', color='#4CAF50'),
        xaxis_title='Year',
        yaxis_title='Total TB Deaths',
        template='plotly_white',
        legend_title='Country',
        font=dict(family='Arial', size=14),
    )

    return map_fig, line_fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
